In [270]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os
import os
os.chdir('..')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [285]:
import h5py
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/pos.mat'
with h5py.File(fname, 'r') as f:
    print (f.keys())
    
    pos = np.array(f['pos']).squeeze()
print (pos.shape)


<KeysViewHDF5 ['pos']>
(37160,)


In [338]:
#################################################
#################################################
#################################################
from scipy.ndimage import gaussian_filter1d
from matplotlib.gridspec import  GridSpec

#
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/binarized_traces.npz'
d = np.load(fname,
              allow_pickle=True)

# 
F_thresholded = d['events_F_threshold_upphase']
oasis_smooth = d['events_Oasis_smooth_upphase']
spikes= d['spikes']

# 
print (spikes.shape)


(1028, 37160)


In [339]:
# 
y_f = []
y_oasis = []

fig=plt.figure()
imgs=[]
for k in range(3):
    
    if k==0:
        data_in = F_thresholded.copy()
        label = 'F_thresholded'

    elif k==1:
        data_in = oasis_smooth.copy()
        label = 'Oasis smoothed x F scaled'
        
    elif k==2: 
        data_in = spikes.copy()
        label = 'spikes'

    ax = plt.subplot(3,1,k+1)
    img = []
    width=1
    for unit in trange(data_in.shape[0]):
    #for unit in np.random.randint(0,data_in.shape[0]):
        #unit=57
        idx = np.where(data_in[unit]>0)[0]
        #print (np.unique(pos[idx]).shape, idx.shape)
        if False:
            y = np.histogram(pos[idx],bins=np.arange(0,180,width))
        else:
            y = np.histogram(pos[idx], weights = data_in[unit][idx],
                             bins=np.arange(0,180,width))

        yy=y[0]/(np.max(y[0])+0.0001)
            
        if k == 0:
            y_f.append(yy)
        elif k==1:
            y_oasis.append(yy)
        img.append(yy)

    # 
    img=np.vstack(img)
    if k==0:
        idx2 = np.argsort(np.argmax(img,axis=1))
        
    img=img[idx2]

    vmax = np.max(img)*.8
    plt.imshow(img,
              aspect='auto',
              cmap='magma',
              vmin=0,
              extent=[0,180,0,data_in.shape[0]],
              vmax=vmax)
    
    plt.ylabel(label + "\nNeuron Ids (ordered by peak time)")
    imgs.append(img)
    
# 
plt.show()




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1028/1028 [00:00<00:00, 4831.12it/s]


In [340]:
#y_f = []
units = [1,43,54,268,272,416,678,820, 500]
ctr=0
fig=plt.figure()
for unit in units:
    ax=plt.subplot(3,3,ctr+1)
    labels = ['F','oasis','0.01cm']
    #clrs = ['lightblue', 'royalblue','blue','lightcoral','pink','red']
    clrs='blue', 'red'
    #for k in range(len(y_f)):
    t=np.linspace(0, 180, y_f[unit].shape[0])
    if False:
        plt.plot(t, y_f[unit]/np.max(y_f[unit]), label=labels[0],c=clrs[0])
        plt.plot(t, y_oasis[unit]/np.max(y_oasis[unit]),label=labels[1],c=clrs[1])
    else:
        plt.plot(t, y_f[unit], label=labels[0],c=clrs[0])
        plt.plot(t, y_oasis[unit],label=labels[1],c=clrs[1])
    plt.legend(fontsize=20)
    #plt.title("Spatial footprint as a function of time bin", fontsize=20)
    #plt.xlabel("Location on track (cm)", fontsize=20)
    plt.xlim(0,180)
    plt.ylim(bottom=0)
    ctr+=1
plt.show()

In [333]:
fig=plt.figure()
ctr=0
k=0
while True:
#for ctr, k in enumerate(range(20,100,1)):
   
    temp1 = y_f[k]
    temp2 = y_oasis[k]

    if np.max(temp1)==0 and np.max(temp2)==0:
        k+=1
        continue
    
    ax=plt.subplot(20,4,ctr+1)
    plt.ylabel(str(k),fontsize=6)
    
    plt.plot(temp1/np.max(temp1),label="F_thresh",c='blue')
    plt.plot(temp2/np.max(temp2), label = "Oasis x F x rolling window",c='red')
    #plt.plot(temp2/np.max(temp2), c='magenta', 
    #         label = "raw spikes")
    #plt.plot(imgs[2][k]/100., label = "spikes")
    plt.xlim(0,180)
    #plt.ylim(bottom=0)

    if ctr==0:
        plt.legend(fontsize=6)
    plt.xticks([])
    plt.yticks([])
    
    
    ctr+=1
    k+=1
    if ctr==80:
        break
        
plt.show()